# 1. Getting the data

### The main code for sending an API request

In [40]:
import requests, json
from secretfile import api_key

origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = "200"
query_string = "thai"

url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_string}&type=restaurant&location={origin_location}&radius={radius_in_meters}&key={api_key}"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
response.raise_for_status()

print(response.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEDY7dPegYtNzt4hkLcXkrC84gN7gyU4QCklJ2iyPkSxA1hPbz7YjK7IPY9WwAFbISRSZ4R_6I7zViunMwgDNMP92V9mgpnDeE6FY00rFmMu-3j2g3CH4ucCrGHLzeUd7lEa4vTGdknJA4MSo3oXK0wpRREXBdRkSaPaT9bc9G1KWnPYqHJM9OgHElg77FAOAlWAtLzYyCfVliHpXC59RDCR54LWA80HqONZ72t6MrtHRfN_y8Opb3MumSPyl2qeWKH3rnvt3DrilMhn5DcoTxhmmDTR4qHOkrRMc4UzEDCO-HECkMsWUs01FvBVUOCkIYkqcnPUclvVwOB8oMyz71lrQL59V4IPdFFUCWGZICc-PyqhGUMjTCB4sweUJ74g4wVSxPWrWxrGUld8Q07dGbPQzPgPQ1vjM7izRpSpKxygavBlHbpPY4kNe_ryhTivTG6bh7uSQyg",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "formatted_address" : "Lilla Kyrkogatan 2, 411 15 Göteborg",
         "geometry" : {
            "location" : {
               "lat" : 57.705267,
               "lng" : 11.965133
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 57.70660272989272,
                  "lng" : 11.96638072989272
               },
               "southwest" : {
           

In [30]:
import pandas as pd

### Formatting the data we just got

We want to write our results to a file so we can make it into a dataform with the help of pandas

In [38]:
with open('restaurants.json', 'w' , encoding='utf-8') as w:
    w.write(response.text)

In [44]:
with open('restaurants.json', 'r', encoding='utf-8') as data_file:
    data = json.load(data_file)
    for restaurant in data['results']:
        print("Name:", restaurant['name'], "Rating:", restaurant['rating'])

Name: Sukhothai Restaurang Rating: 4.1
Name: S.p.i.c.e Rating: 3.7
Name: Niyom Thai Rating: 3.6
Name: Ullevi Thaikök Rating: 4.5
Name: Samui Thai Kitchen Rating: 4
Name: Restaurang Thai Nisa , Mintensgatans Thai Take Away AB Rating: 3.5
Name: Arojj Dii Rating: 4.4
Name: Moon Thai Kitchen Rating: 4.4
Name: Toui ́s Thai Kök Vasastan Rating: 4.3
Name: Thai Prayook Rating: 3.9
Name: Ching Palace Rating: 3.8
Name: Totto's Thai Food & Take Away AB Rating: 4.1
Name: Thai Spice Rating: 4
Name: Thai Family Kitchen Rating: 4.2
Name: Tawan Thai Kitchen Rating: 4.3
Name: Bangkok Kitchen Göteborg Rating: 4.2
Name: Thaimums Take Away Rating: 4.3
Name: Sweet N' Spicy Thai Kitchen Rating: 3.7
Name: Mae Krua Rating: 4
Name: Lai Thai Restaurang Rating: 4
